In [1]:
import pandas as pd
import numpy as np
import time
import re

In [2]:
df = pd.read_csv('temp/glassdoor refined.csv')

In [3]:
df.head()

,Company,Location,Title,Rating,Salary,Description,Founded,Size,Industry,Sector,Type,Revenue
0,Purchasing Power\n3.2,"Atlanta, GA",Data Scientist,3.2,$66K - $112K (Glassdoor est.),Data Scientist\nLocation\n\n\nAtlanta - Midtow...,2001.0,201 to 500 Employees,Financial Transaction Processing,Finance,Company - Private,$100 to $500 million (USD)
1,Varen Technologies\n4.2,"Annapolis Junction, MD",Data Scientist,4.2,$76K - $111K (Glassdoor est.),"At Varen, our performance is measured by the s...",2005.0,51 to 200 Employees,Enterprise Software & Network Solutions,Information Technology,Company - Private,$25 to $50 million (USD)
2,Big Fish Games\n3.2,"Oakland, CA",Senior Data Scientist,3.2,$109K - $175K (Glassdoor est.),Job Posting Title\nSenior Data Scientist\nSumm...,2002.0,501 to 1000 Employees,Video Games,Media,Subsidiary or Business Segment,$50 to $100 million (USD)
3,GEICO\n3.5,"Chevy Chase, MD",Principal Data Scientist,3.5,$104K - $170K (Glassdoor est.),"Working out of our Chevy Chase, MD/Washington ...",1936.0,10000+ Employees,Insurance Carriers,Insurance,Subsidiary or Business Segment,$10+ billion (USD)
4,Stanley Black & Decker\n3.5,"Fishers, IN",AI Data Scientist,3.5,$74K - $130K (Glassdoor est.),About Stanley Black & Decker\n\nJoining the St...,1843.0,10000+ Employees,Industrial Manufacturing,Manufacturing,Company - Public,$10+ billion (USD)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      884 non-null    object 
 1   Location     883 non-null    object 
 2   Title        883 non-null    object 
 3   Rating       804 non-null    float64
 4   Salary       884 non-null    object 
 5   Description  883 non-null    object 
 6   Founded      700 non-null    float64
 7   Size         824 non-null    object 
 8   Industry     748 non-null    object 
 9   Sector       748 non-null    object 
 10  Type         824 non-null    object 
 11  Revenue      824 non-null    object 
dtypes: float64(2), object(10)
memory usage: 83.0+ KB


In [5]:
df.duplicated().sum()

0

In [6]:
df.dropna(subset = ['Salary', 'Company', 'Location'], inplace = True)
df.drop_duplicates(inplace = True)
df.reset_index(drop = True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      883 non-null    object 
 1   Location     883 non-null    object 
 2   Title        883 non-null    object 
 3   Rating       804 non-null    float64
 4   Salary       883 non-null    object 
 5   Description  883 non-null    object 
 6   Founded      700 non-null    float64
 7   Size         824 non-null    object 
 8   Industry     748 non-null    object 
 9   Sector       748 non-null    object 
 10  Type         824 non-null    object 
 11  Revenue      824 non-null    object 
dtypes: float64(2), object(10)
memory usage: 82.9+ KB


## Salary parsing

In [7]:
per_hour = df['Salary'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
employer = df['Salary'].apply(lambda x: 1 if 'employer' in x.lower() else 0)

df['Hourly'] = per_hour
df['Employer Provided'] = employer

In [8]:
def salary_parse(salary):
    salary_split = salary.split('(')[0]
    if salary_split.endswith('r'):
        salary_split = salary.split(' Per')[0].split('-')
        salary_split = [int(i.replace('$', '')) for i in salary_split]
        salary_split = [i * 40 * 50 for i in salary_split]
    else:
        salary_split = salary.split(' (')[0].split(' - ')
        salary_split = [int(i.replace('$', '').replace('K', '')) for i in salary_split]
        salary_split = [i * 1000 for i in salary_split]
    return salary_split

In [9]:
df['Low Salary'] = df['Salary'].apply(salary_parse).apply(lambda x: x[0])
df['High Salary'] = df['Salary'].apply(salary_parse).apply(lambda x: x[1])
df['Average Salary'] = ((df['Low Salary'] + df['High Salary']) / 2).astype(int)
df[['Salary', 'Hourly', 'Employer Provided', 'Low Salary', 'High Salary',\
    'Average Salary']].sample(10)

,Salary,Hourly,Employer Provided,Low Salary,High Salary,Average Salary
622,$108K - $175K (Glassdoor est.),0,0,108000,175000,141500
12,$46K - $83K (Glassdoor est.),0,0,46000,83000,64500
881,$74K - $126K (Glassdoor est.),0,0,74000,126000,100000
655,$121K - $191K (Glassdoor est.),0,0,121000,191000,156000
285,$65K - $110K (Glassdoor est.),0,0,65000,110000,87500
459,$55K - $94K (Glassdoor est.),0,0,55000,94000,74500
340,$74K - $128K (Glassdoor est.),0,0,74000,128000,101000
185,$58K - $105K (Glassdoor est.),0,0,58000,105000,81500
416,$69K - $117K (Glassdoor est.),0,0,69000,117000,93000
481,$162K - $269K (Glassdoor est.),0,0,162000,269000,215500


In [10]:
df['Hourly'].value_counts(), df['Employer Provided'].value_counts()

(0    883
 Name: Hourly, dtype: int64,
 0    851
 1     32
 Name: Employer Provided, dtype: int64)

## City & State parsing

In [11]:
for i in df.index:
    if len(df.loc[i,'Location'].split(', ')) == 3:
        df.loc[i, 'City'] = df.loc[i, 'Location'].split(', ')[1]
    elif len(df.loc[i,'Location'].split(', ')) == 2:
        df.loc[i, 'City'] = df.loc[i, 'Location'].split(', ')[0]
    else:
        df.loc[i, 'City'] = np.nan

In [12]:
place = df['Location'].apply(lambda x: x.split(', '))

for i in range(len(place)):
    if len(place.loc[i]) == 1:
        place.loc[i].append(np.nan)

df['State'] = place.apply(lambda x: x[1])

In [13]:
df = df[df['Location'] != 'United States']
df.loc[df['Location'] == 'Remote', 'State'] = 'Remote'
df.loc[df['Location'] == 'Remote', 'City'] = 'Remote'

In [14]:
states = pd.read_csv('temp/us-states.csv', sep = ' - ', engine = 'python', header = None,
                     names = ['State', 'Abbreviation'])

for i in df.index:
    if df.loc[i, 'Location'] in list(states['State'].unique()):
        state_full = df.loc[i, 'Location']
        state_abbrev = list(states[states['State'] == state_full]['Abbreviation'])[0]
        df.loc[i, 'State'] = state_abbrev

In [15]:
df[['Location', 'City', 'State']].sample(10)

,Location,City,State
93,"San Mateo, CA",San Mateo,CA
555,"Philadelphia, PA",Philadelphia,PA
383,"New York, NY",New York,NY
721,"Woburn, MA",Woburn,MA
5,"Rogers, AR",Rogers,AR
612,"New York, NY",New York,NY
179,"Houston, TX",Houston,TX
862,"Watertown, MA",Watertown,MA
360,"Cranston, RI",Cranston,RI
434,"Houston, TX",Houston,TX


In [16]:
df['State'].value_counts()

CA             165
VA             114
NY              68
TX              55
WA              50
MA              43
MD              36
IL              31
NJ              31
NC              25
PA              22
DC              22
MO              22
Remote          20
OH              19
FL              17
GA              16
CO              16
MN              14
MI              12
TN               8
CT               8
AZ               7
KY               6
OR               4
NM               3
UT               3
AL               3
SC               3
ID               2
KS               2
AR               2
WI               2
IN               2
NV               2
IA               2
RI               2
Los Angeles      2
ME               1
NH               1
PR               1
NE               1
Fulton           1
Name: State, dtype: int64

## Company name text only

In [17]:
df['Company'] = df['Company'].apply(lambda x: x.split('\n')[0])
df[['Company', 'Rating']].sample(10)

,Company,Rating
801,Absolute Business Solutions Corp,3.1
45,pulseData,5.0
524,AptoNet Inc,4.9
832,NYSTEC,4.0
499,Scale Media,4.8
340,Ursus,4.4
443,J.P. Morgan,4.0
534,"DirectViz Solutions, LLC",3.3
686,Klaviyo,4.8
211,S2Technologies LLC,5.0


In [18]:
df['Company'].nunique()

585

## Age of company

In [19]:
df['Founded'].fillna(-1, inplace = True)
df['Founded'] = df['Founded'].astype(int)

for i in df.index:
    if df.loc[i, 'Founded'] == -1:
        df.loc[i, 'Age'] = -1
    else:
        df.loc[i, 'Age'] = int(time.localtime()[0]) - df.loc[i, 'Founded']

df['Age'] = df['Age'].astype(int)
df[['Company', 'Founded', 'Age']].sample(10)

,Company,Founded,Age
866,Analog Devices,1965,56
393,Sam's Club,1983,38
124,Vytalize Health,-1,-1
556,Practice Paradigm,-1,-1
861,Ascend Innovations,1997,24
289,Intuit - Data,1983,38
342,APR Consulting Inc,1980,41
364,Envision LLC,1994,27
515,Logic20/20,2005,16
546,Riverside Health System,1915,106


## Parsing of job description

In [20]:
jobs = {'Python' : 'python', 'R' : ['rstudio', 'r studio', 'r-studio'], 'SQL' : 'sql',\
        'ML' : 'machine learning', 'DL' : 'deep learning', 'Excel' : 'excel', 'Spark' : 'spark',\
        'AWS' : 'aws', 'BI' : ['powerbi', 'power bi', 'power-bi', 'tableau'],\
        'Viz' : ['Data Visualization', 'data visualization', 'visualization']}

jobs_match = []
for key, value in jobs.items():
    if type(jobs[key]) != list:
        jobs_match.append(re.compile(jobs[key]))
    else:
        jobs_match.append(re.compile('|'.join(jobs[key])))

for i in df.index:
    j = 0
    while j < len(jobs.keys()):
        for key,value in jobs.items():
            if jobs_match[j].search(df.loc[i, 'Description'].lower()):
                df.loc[i,key] = 1
            else:
                df.loc[i,key] = 0
            j += 1
        
cols = jobs.keys()
for i in cols:
    df[i] = df[i].astype(int)

df[['Description', 'Python', 'R', 'SQL', 'ML', 'DL', 'Excel', 'Spark', 'AWS', 'BI', 'Viz']].sample(10)

,Description,Python,R,SQL,ML,DL,Excel,Spark,AWS,BI,Viz
123,Quantitative Analyst / Data Scientist\nMaple G...,0,0,0,0,0,0,0,0,0,0
813,Strength Through Diversity\nGround breaking sc...,0,0,0,0,0,0,0,0,0,1
340,Job Title: Data Scientist - Big Data\nLocation...,1,0,0,1,0,0,0,0,1,1
69,ABOUT POWER FACTORS\n\nPower Factors is a lead...,0,0,0,0,0,0,0,0,0,0
810,"Data Scientist\nLocation\n\n\nNorfolk, VA\n\nJ...",0,0,0,0,0,0,0,0,0,0
178,"Senior Data Scientist\nCincinnati, Ohio\n$130,...",0,0,0,0,0,0,0,0,0,0
150,JOB DESCRIPTION\n\nLIFE AT IMC AS DATA ENGINEE...,0,0,0,0,0,0,0,0,0,0
622,Our customers are inundated with information f...,0,0,0,0,0,0,0,0,0,0
434,Company: IBERDROLA SOLUTIONS LLC\nLocation:Hou...,0,0,0,0,0,0,0,0,0,0
359,Job Summary\nThe Cooperative Institute for Res...,0,0,0,0,0,0,0,0,0,0


In [21]:
df['SQL'].value_counts(), df['R'].value_counts(), df['Python'].value_counts(), df['ML'].value_counts(),\
df['DL'].value_counts(), df['Excel'].value_counts(), df['Spark'].value_counts(), df['AWS'].value_counts(),\
df['BI'].value_counts()

(0    765
 1    102
 Name: SQL, dtype: int64,
 0    864
 1      3
 Name: R, dtype: int64,
 0    720
 1    147
 Name: Python, dtype: int64,
 0    582
 1    285
 Name: ML, dtype: int64,
 0    837
 1     30
 Name: DL, dtype: int64,
 0    743
 1    124
 Name: Excel, dtype: int64,
 0    819
 1     48
 Name: Spark, dtype: int64,
 0    802
 1     65
 Name: AWS, dtype: int64,
 0    828
 1     39
 Name: BI, dtype: int64)

In [22]:
df.to_csv('temp/glassdoor cleaned', index = False)

In [23]:
df = pd.read_csv('temp/glassdoor cleaned')
df.sample(10)

,Company,Location,Title,Rating,Salary,Description,Founded,Size,Industry,Sector,...,Python,R,SQL,ML,DL,Excel,Spark,AWS,BI,Viz
70,"Culmen International, LLC","Arlington, VA",Data Scientist,3.8,$62K - $105K (Glassdoor est.),"Data Scientist\nArlington, VA 20301\n\nSecurit...",2004,201 to 500 Employees,Federal Agencies,Government,...,1,0,0,0,0,1,0,0,1,1
192,Lawrence Berkeley National Laboratory,"San Francisco, CA",DESI Data Management Project Scientist,4.4,$58K - $105K (Glassdoor est.),Berkeley Lab’s Physics Division has an opening...,1931,1001 to 5000 Employees,State & Regional Agencies,Government,...,0,0,0,0,0,0,0,0,0,0
313,Apple,"Santa Clara, CA","Apple One Business Data Scientist, Apple Media...",4.3,$79K - $130K (Glassdoor est.),"Posted: Sep 25, 2020\nRole Number:\n200195438\...",1976,10000+ Employees,Computer Hardware & Software,Information Technology,...,0,0,0,0,0,0,0,0,0,0
551,stanleyreid,"McLean, VA","Data Scientist - FS Poly, to 200K+",NaN,$110K - $171K (Glassdoor est.),Our client is leading an effort to replace a l...,-1,NaN,NaN,NaN,...,1,0,0,1,0,0,0,1,0,0
653,MCG Health,Remote,Data Engineer (Automation),4.7,$95K - $162K (Glassdoor est.),We are a company of passionate individuals str...,1988,201 to 500 Employees,Health Care Services & Hospitals,Health Care,...,0,0,0,0,0,0,0,0,0,0
493,I.M. Systems Group,"College Park, MD",NOA2107 Support Scientist Satellite Data Assim...,3.6,$162K - $269K (Glassdoor est.),I.M. Systems Group\n\nLocation: US-MD-College ...,1986,201 to 500 Employees,Consulting,Business Services,...,0,0,0,0,0,0,0,0,0,0
545,Children's Hospital of Philadelphia,"Philadelphia, PA",Data Scientist II - 64037,3.7,$110K - $171K (Glassdoor est.),Job Type:\nLOC_ROBERTS-Roberts Ctr Pediatric R...,1855,1001 to 5000 Employees,Health Care Services & Hospitals,Health Care,...,0,0,0,1,0,0,0,0,0,0
204,TSG Engineering,"Ellicott City, MD","Data Scientist/Analyst [R, Python,Google Cloud]",NaN,$58K - $105K (Glassdoor est.),Benefits\nTSG Engineering offers a competitive...,-1,Unknown,NaN,NaN,...,1,0,0,1,0,0,1,1,0,0
210,Lawrence Berkeley National Laboratory,"San Francisco, CA",DESI Data Management Project Scientist,4.4,$80K - $136K (Glassdoor est.),Berkeley Lab’s Physics Division has an opening...,1931,1001 to 5000 Employees,State & Regional Agencies,Government,...,0,0,0,0,0,0,0,0,0,0
235,PenFed Credit Union,"McLean, VA",Senior Data Scientist Credit Modeling,3.4,$80K - $136K (Glassdoor est.),Overview\nAre you looking to take your career ...,1935,1001 to 5000 Employees,Banks & Credit Unions,Finance,...,0,0,0,0,0,0,0,0,0,0
